# Backtest of Technical Analysis Based Strategies | DMAC
---

__Authors:__ Emily Bertani, Max Acheson, Josh Mischung  
__Data Source:__ Yahoo Finance accessed by `pandas-datareader` and `yfinance`  
__Strategy:__ DMAC 
__Instrument(s):__ SPY  

__Intended Use:__  
The primary objective of this notebook is to determine the profitability and optimum window sizes of the DMAC strategy.

<br>

## Imports & Functions
---
__Imports__

In [5]:
# Supress bokeh warnings generated by backtesting
import warnings
warnings.filterwarnings('ignore')

import backtesting

In [14]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
import pandas_datareader.data as pdr
import yfinance as yf

<br>

__Functions & Classes__

In [15]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()


class SmaCross(Strategy):
    n1=10
    n2=20
    
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, self.data.Close, self.n1)
        self.ma2 = self.I(SMA, self.data.Close, self.n2)
        
    def next(self):
        if crossover(self.ma1, self.ma2):
            self.position.close()
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.position.close()
            self.sell()

<br>

## Retreive and Format Data
---

In [13]:
# Set variables
ticker = 'AAPL'
start_date = '2015-05-01'
end_date = '2022-03-25'

# Create OHLCV DataFrame
stock_df = pdr.get_data_yahoo(
    ticker,
    start_date,
    end_date
)

stock_df.drop(columns=['Close'], inplace=True)
stock_df.rename(columns={'Adj Close' : 'Close'}, inplace=True)
stock_df.insert(3, 'Close', stock_df.pop('Close'))

stock_df.head()

,High,Low,Open,Close,Volume
Date,,,,,
2015-05-01,32.532501,31.325001,31.525000,29.264734,234050400.0
2015-05-04,32.642502,32.064999,32.375000,29.207998,203953200.0
2015-05-05,32.112499,31.445000,32.037498,28.549854,197085600.0
2015-05-06,31.687500,30.840000,31.639999,28.370567,288564000.0
2015-05-07,31.520000,31.004999,31.192499,28.546047,175763600.0


In [16]:
bt = Backtest(stock_df, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats

Start                     2015-05-01 00:00:00
End                       2022-03-25 00:00:00
Duration                   2520 days 00:00:00
Exposure Time [%]                   98.389879
Equity Final [$]                 28677.129808
Equity Peak [$]                  42434.075594
Return [%]                         186.771298
Buy & Hold Return [%]              497.032591
Return (Ann.) [%]                   16.493553
Volatility (Ann.) [%]               47.715907
Sharpe Ratio                         0.345662
Sortino Ratio                         0.62189
Calmar Ratio                         0.375381
Max. Drawdown [%]                  -43.938146
Avg. Drawdown [%]                   -6.920735
Max. Drawdown Duration      497 days 00:00:00
Avg. Drawdown Duration       48 days 00:00:00
# Trades                                   67
Win Rate [%]                        40.298507
Best Trade [%]                      79.788108
Worst Trade [%]                    -14.188268
Avg. Trade [%]                    

In [17]:
bt.plot();